In [9]:
import pandas as pd
import json

In [10]:
#get original data
videos = pd.read_json('./Data/video_metadata.jsonl',lines=True,encoding='utf-8')
with open('./Data/categories.json') as file:
    categories = json.load(file)
channels = pd.read_json('./Data/channel_metadata.jsonl',lines=True,encoding='utf-8')
comments = pd.read_json('./Data/comments.jsonl',lines=True,encoding='utf-8')

In [12]:
videos_from_channels= pd.read_json('./Data/video_metadata_from_channels.jsonl',lines=True,encoding='utf-8')
unified_videos = pd.concat([videos,videos_from_channels])
unified_videos = unified_videos[unified_videos['category']!='from_channel']
unified_videos = unified_videos[unified_videos['category'].astype(int)<30].drop_duplicates('videoId')
with open('./Data/complete_videos_metadata.jsonl','w',encoding='utf-8') as file:
    for line in unified_videos.to_dict(orient='records'):
        json.dump(line,file,ensure_ascii=False)
        file.write('\n')

In [13]:
#exclude undesirable categories are no category and any with id higher than movies
comments = pd.merge(comments,unified_videos[['videoId','category']],on='videoId')#.to_dict(orient='records')
comments = comments.drop_duplicates('id').to_dict(orient='records')

In [15]:
with open('./Data/complete_comments.jsonl','w',encoding='utf-8') as file:
    for line in comments:
        json.dump(line,file,ensure_ascii=False)
        file.write('\n')

In [19]:
for chunk in pd.read_json('./Data/comments_from_channels.jsonl',encoding='utf-8',chunksize=10**6,lines=True):
    chunk = pd.merge(chunk,unified_videos[['videoId','category']],on='videoId')
    chunk = chunk.drop_duplicates('id').to_dict(orient='records')
    with open('./Data/complete_comments.jsonl','a',encoding='utf-8') as file:
        for line in chunk:
            json.dump(line,file,ensure_ascii=False)
            file.write('\n')